# 설치 & 경로/설정

In [ ]:
# (필요 시) 설치
%pip -q install transformers pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
# 파일 경로
IN_TXT  = "../day1/samples_pre.txt"
OUT_CSV = "./bert_inputs.csv"
OUT_META= "./tokenizer_meta.json"
OUT_STAT= "./stats.txt"

# 토크나이저/길이 설정
MODEL_NAME = "bert-base-uncased"   # 필요시 distilbert-base-uncased 등
MAX_LEN    = 320                   # 권장: 256~384 범위
PAD        = "max_length"
TRUNC      = "longest_first"
ADD_SPE    = True                  # [CLS]/[SEP] 자동

## bert-base-uncased
- bert
	- 모델 구조가 BERT (Bidirectional Encoder Representations from Transformers) 임을 의미.
	- 구글이 2018년에 발표한 대표적인 Transformer 기반 언어모델.
	- 특징: 문장을 양방향으로 동시에 보면서 문맥을 이해하는 능력.

- base
	- 모델의 크기(scale).
	- bert-base: 12 layers (transformer blocks), hidden size=768, attention heads=12, 파라미터 약 110M.
	- bert-large: 24 layers, hidden size=1024, attention heads=16, 파라미터 약 340M.

- uncased
	- "대소문자 구분을 하지 않는다"는 의미.
	- 토큰화할 때 모든 단어를 소문자로 변환하고, 사전도 소문자 기반으로 구성됨.

### bert-base-uncased 사양 요약
- Layers: 12 (Transformer Encoder blocks)
- Hidden size: 768
- Attention heads: 12
- Parameters: 약 110M
- Vocabulary size: 30,522 (WordPiece 토크나이저)
- 학습 데이터: BookCorpus (8억 단어) + Wikipedia (25억 단어)
- 특수 토큰: [CLS], [SEP], [PAD], [MASK], [UNK] 포함

# 2. 입력 로드(빈 줄 제거) + 간단 통계

In [9]:
from pathlib import Path
import numpy as np

# 입력 읽기
raw_lines = Path(IN_TXT).read_text(encoding="utf-8").splitlines()
print("raw_lines: ", raw_lines)
texts = [ln.strip() for ln in raw_lines if ln.strip()]
print("texts: ", texts)
print(f"Loaded {len(texts)} lines from {IN_TXT}")

# 토큰 수(스페이스 분할 기준) 대략 통계
tok_counts = [len(t.split()) for t in texts]
if tok_counts:
    a = np.array(tok_counts)
    print(f"Tokens per line (space-split) -> mean={a.mean():.1f}, median={np.median(a):.0f}, max={a.max()}")
else:
    print("WARNING: no texts loaded.")


raw_lines:  ['sep_path get /wp B#NUM php sep_q file sep_h host example com sep_ua ua mozilla/#NUM #NUM charon inferno sep_body body len #NUM body sig', 'sep_path resp sep_h h co application/x msdownload h co gzip h tr chunked sep_body body len #NUM body sig mz']
texts:  ['sep_path get /wp B#NUM php sep_q file sep_h host example com sep_ua ua mozilla/#NUM #NUM charon inferno sep_body body len #NUM body sig', 'sep_path resp sep_h h co application/x msdownload h co gzip h tr chunked sep_body body len #NUM body sig mz']
Loaded 2 lines from ../day1/samples_pre.txt
Tokens per line (space-split) -> mean=21.5, median=22, max=23


# Tokenizer 로드

In [ ]:
%pip unistall huggingface_hub 
%pip install huggingface_hub==0.34.4

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

meta = {
    "model_name_or_path": MODEL_NAME,
    "vocab_size": tokenizer.vocab_size,
    "max_length": MAX_LEN,
    "padding": PAD,
    "truncation": TRUNC,
    "add_special_tokens": ADD_SPE,
    "do_lower_case": getattr(tokenizer, "do_lower_case", None),
    "special_tokens_map": tokenizer.special_tokens_map
}
meta


{'model_name_or_path': 'bert-base-uncased',
 'vocab_size': 30522,
 'max_length': 320,
 'padding': 'max_length',
 'truncation': 'longest_first',
 'add_special_tokens': True,
 'do_lower_case': True,
 'special_tokens_map': {'unk_token': '[UNK]',
  'sep_token': '[SEP]',
  'pad_token': '[PAD]',
  'cls_token': '[CLS]',
  'mask_token': '[MASK]'}}

# 배치 토크나이징 → input_ids / attention_mask 생성 + 스팟체크

In [17]:
enc = tokenizer(
    texts,
    add_special_tokens=ADD_SPE,
    return_attention_mask=True,
    padding=PAD,
    truncation=True,                 # longest_first는 pair에서 의미; 단일은 True로 충분
    max_length=MAX_LEN
)
input_ids = enc["input_ids"]            # List[List[int]] (N×L)
attention_mask = enc["attention_mask"]  # List[List[int]] (N×L)

N = len(texts)
L = len(input_ids[0]) if N else 0
print(f"Shapes -> input_ids: {len(input_ids)}×{L}, attention_mask: {len(attention_mask)}×{L}")
print(input_ids[0])
# 스팟체크: 0번째 문장 복원/마스크 합계
if N:
    print("decode[0]:", tokenizer.decode(input_ids[0], skip_special_tokens=True)[:200])
    print("mask_sum[0]:", sum(attention_mask[0]))


Shapes -> input_ids: 2×320, attention_mask: 2×320
[101, 19802, 1035, 4130, 2131, 1013, 1059, 2361, 1038, 1001, 16371, 2213, 25718, 19802, 1035, 1053, 5371, 19802, 1035, 1044, 3677, 2742, 4012, 19802, 1035, 25423, 25423, 9587, 5831, 4571, 1013, 1001, 16371, 2213, 1001, 16371, 2213, 25869, 2239, 21848, 19802, 1035, 2303, 2303, 18798, 1001, 16371, 2213, 2303, 9033, 2290, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# CSV/메타/통계 저장

In [18]:
import json, pandas as pd

def arr_to_str(arr):
    return " ".join(str(x) for x in arr)

rows = []
for t, ids, m in zip(texts, input_ids, attention_mask):
    rows.append({
        "text": t,
        "input_ids": arr_to_str(ids),
        "attention_mask": arr_to_str(m),
    })

df = pd.DataFrame(rows, columns=["text","input_ids","attention_mask"])
display(df)
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
print(f"Saved CSV -> {OUT_CSV} (rows={len(df)})")

with open(OUT_META, "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)
print(f"Saved meta -> {OUT_META}")

# 간단 통계
mask_sums = [sum(m) for m in attention_mask] if attention_mask else []
with open(OUT_STAT, "w", encoding="utf-8") as f:
    f.write(f"lines={len(texts)}\n")
    if mask_sums:
        f.write(f"attn_sum_mean={float(np.mean(mask_sums)):.1f}\n")
        f.write(f"attn_sum_median={float(np.median(mask_sums)):.1f}\n")
        f.write(f"attn_sum_max={int(np.max(mask_sums))}\n")
print(f"Saved stats -> {OUT_STAT}")


,text,input_ids,attention_mask
0,sep_path get /wp B#NUM php sep_q file sep_h ho...,101 19802 1035 4130 2131 1013 1059 2361 1038 1...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
1,sep_path resp sep_h h co application/x msdownl...,101 19802 1035 4130 24501 2361 19802 1035 1044...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...


Saved CSV -> ./bert_inputs.csv (rows=2)
Saved meta -> ./tokenizer_meta.json
Saved stats -> ./stats.txt


### 커스텀 토크나이저
- 기존 BERT 토크나이저에 토큰만 추가
``` python
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 1) 추가할 토큰 정의
new_tokens = ["[URL]", "[IP]", "cmd.exe", "powershell"]  # 예시
num_added = tokenizer.add_tokens(new_tokens, special_tokens=False)

# 2) 모델 로드 + 임베딩 크기 리사이즈
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=13)
if num_added > 0:
    model.resize_token_embeddings(len(tokenizer))  # 새 토큰 임베딩은 무작위 초기화

# (옵션) 저장
tokenizer.save_pretrained("./ckpt_tok_custom")
model.save_pretrained("./ckpt_tok_custom")
```

### 커스텀 토크나이저
- 기존 BERT 토크나이저에 토큰만 추가
``` python
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 1) 추가할 토큰 정의
new_tokens = ["[URL]", "[IP]", "cmd.exe", "powershell"]  # 예시
num_added = tokenizer.add_tokens(new_tokens, special_tokens=False)

# 2) 모델 로드 + 임베딩 크기 리사이즈
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=13)
if num_added > 0:
    model.resize_token_embeddings(len(tokenizer))  # 새 토큰 임베딩은 무작위 초기화

# (옵션) 저장
tokenizer.save_pretrained("./ckpt_tok_custom")
model.save_pretrained("./ckpt_tok_custom")
```

- 커스텀 토크나이저
	1. 기존 bert-base-uncased 가중치를 부분 로드(임베딩만 새로 초기화)
		- 장점: 나머지 트랜스포머 블록 가중치를 최대한 재사용
		- 주의: 토큰화가 달라져도 BERT 내부는 그대로라 초기 성능이 다소 흔들릴 수 있음
		- 토크나이저 학습 (예: tokenizers 또는 sentencepiece)
		- 학습된 vocab/merges로 BertTokenizerFast 생성
		- 모델은 bert-base-uncased에서 불러오되, 임베딩 사이즈 불일치는 무시하고 새로 리사이즈
```python
# 2-A-1) (예시) tokenizers로 WordPiece/BPE 학습
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors
from pathlib import Path

files = ["corpus1.txt", "corpus2.txt"]  # 도메인 코퍼스
tokenizer_tr = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer_tr.pre_tokenizer = pre_tokenizers.Whitespace()
trainer = trainers.WordPieceTrainer(vocab_size=32000, special_tokens=["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"])
tokenizer_tr.train(files, trainer)
Path("my_vocab.json").write_text(tokenizer_tr.to_str(), encoding="utf-8")  # 또는 tokenizer_tr.save("tokenizer.json")

# 2-A-2) HF 토크나이저로 래핑
from transformers import BertTokenizerFast, AutoModel
# WordPiece라면 vocab.txt 형태가 더 흔합니다 (tokenizers JSON → vocab.txt 변환 필요)
# 여기서는 vocab.txt가 준비돼 있다고 가정
tok = BertTokenizerFast(vocab_file="vocab.txt", do_lower_case=True)

# 2-A-3) 모델 로드 (임베딩 불일치 허용)
model = AutoModel.from_pretrained(
    "bert-base-uncased",
    ignore_mismatched_sizes=True
)

# 2-A-4) 임베딩 리사이즈
model.resize_token_embeddings(len(tok))

# 저장
tok.save_pretrained("./ckpt_tok_custom_full")
model.save_pretrained("./ckpt_tok_custom_full")
```

	2. 새 config로 BERT를 새로 초기화
		- 장점: 완전한 자유도 (vocab 크기/스페셜 토큰/최대 길이 등)
		- 단점: 사전학습이 없으면 성능이 매우 낮음 → 대규모 프리트레이닝 필요
```python
from transformers import BertConfig, BertForMaskedLM, BertTokenizerFast

tok = BertTokenizerFast(vocab_file="vocab.txt", do_lower_case=True)

config = BertConfig(
    vocab_size=len(tok),
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    max_position_embeddings=512
)
model = BertForMaskedLM(config)  # 랜덤 초기화

# → 대규모 MLM 프리트레이닝이 필요
```